**ADK Notebook for Academic Research**
https://github.com/google/adk-samples/tree/main/python/agents/academic-research

In [2]:
!pip install poetry


In [3]:
import os

!git clone https://github.com/google/adk-samples.git

os.chdir("/content/adk-samples/python/agents/academic-research")
print("Now in:", os.getcwd())

!pwd
!ls

fatal: destination path 'adk-samples' already exists and is not an empty directory.
Now in: /content/adk-samples/python/agents/academic-research
/content/adk-samples/python/agents/academic-research
 academic_research	 eval				     README.md
 academic-research.svg	'Karimian et al. 2024 Science.pdf'   tests
 deployment		 pyproject.toml


In [ ]:
!poetry install

Creating virtualenv academic-research-d-srpqT4-py3.11 in /root/.cache/pypoetry/virtualenvs
Updating dependencies
Resolving dependencies... (12.9s)

Package operations: 103 installs, 0 updates, 0 removals

  - Installing zipp (3.23.0): Pending...
  - Installing zipp (3.23.0): Installing...
  - Installing zipp (3.23.0)
  - Installing importlib-metadata (8.7.0): Pending...
  - Installing protobuf (6.32.1): Pending...
  - Installing pyasn1 (0.6.1): Pending...
  - Installing typing-extensions (4.15.0): Pending...
  - Installing pyasn1 (0.6.1): Pending...
  - Installing typing-extensions (4.15.0): Pending...
  - Installing protobuf (6.32.1): Installing...
  - Installing pyasn1 (0.6.1): Pending...
  - Installing typing-extensions (4.15.0): Pending...
  - Installing pyasn1 (0.6.1): Pending...
  - Installing typing-extensions (4.15.0): Pending...
  - Installing protobuf (6.32.1)
  - Installing pyasn1 (0.6.1): Pending...
  - Installing typing-extensions (4.15.0): Pending...
  - Installing protob

In [4]:
import os

os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "true"
os.environ["GOOGLE_CLOUD_PROJECT"] = "instr-cs795-fall25-hqin-1"
os.environ["GOOGLE_CLOUD_LOCATION"] = "us-central1"
os.environ["GOOGLE_CLOUD_STORAGE_BUCKET"] = "instr-cs795-fall25-hqin-1-tstilwel"


In [5]:
import google.auth
creds, project = google.auth.default()
print("Project:", project)

Project: instr-cs795-fall25-hqin-1


In [ ]:
import os
from vertexai import agent_engines
from vertexai.preview.reasoning_engines import AdkApp
import vertexai

# Your repo package
from academic_research.agent import root_agent

project = os.environ["GOOGLE_CLOUD_PROJECT"]
location = os.environ["GOOGLE_CLOUD_LOCATION"]
bucket = os.environ["GOOGLE_CLOUD_STORAGE_BUCKET"]

vertexai.init(project=project, location=location, staging_bucket=f"gs://{bucket}")

# CREATE (returns a remote agent with resource name)
adk_app = AdkApp(agent=root_agent, enable_tracing=True)
remote_agent = agent_engines.create(
    adk_app,
    display_name=root_agent.name,
    requirements=[
        "google-adk (>=0.0.2)",
        "google-cloud-aiplatform[agent_engines] (>=1.91.0,!=1.92.0)",
        "google-genai (>=1.5.0,<2.0.0)",
        "pydantic (>=2.10.6,<3.0.0)",
        "absl-py (>=2.2.1,<3.0.0)",
    ],
)
print("Created:", remote_agent.resource_name)
resource_id = remote_agent.resource_name  # save this string


INFO:vertexai.agent_engines:Identified the following requirements: {'pydantic': '2.11.7', 'google-cloud-aiplatform': '1.108.0', 'cloudpickle': '3.1.1'}
INFO:vertexai.agent_engines:The following requirements are appended: {'cloudpickle==3.1.1'}
INFO:vertexai.agent_engines:The final list of requirements: ['google-adk (>=0.0.2)', 'google-cloud-aiplatform[agent_engines] (>=1.91.0,!=1.92.0)', 'google-genai (>=1.5.0,<2.0.0)', 'pydantic (>=2.10.6,<3.0.0)', 'absl-py (>=2.2.1,<3.0.0)', 'cloudpickle==3.1.1']
INFO:vertexai.agent_engines:Using bucket instr-cs795-fall25-hqin-1-tstilwel
INFO:vertexai.agent_engines:Wrote to gs://instr-cs795-fall25-hqin-1-tstilwel/agent_engine/agent_engine.pkl
INFO:vertexai.agent_engines:Writing to gs://instr-cs795-fall25-hqin-1-tstilwel/agent_engine/requirements.txt
INFO:vertexai.agent_engines:Creating in-memory tarfile of extra_packages
INFO:vertexai.agent_engines:Writing to gs://instr-cs795-fall25-hqin-1-tstilwel/agent_engine/dependencies.tar.gz
INFO:vertexai.agent

Created: projects/104115398803/locations/us-central1/reasoningEngines/8912856759031300096


In [6]:
from vertexai import agent_engines
agents = agent_engines.list()
for a in agents:
    print(a.name, a.display_name, a.create_time)


8912856759031300096 academic_coordinator 2025-10-07 13:46:35.052653+00:00


In [8]:
from vertexai import agent_engines

resource_id = "projects/instr-cs795-fall25-hqin-1/locations/us-central1/reasoningEngines/8912856759031300096"
user_id = "colab_user_1"
prompt = "Find two recent review papers on telomere biology and summarize key trends."

agent = agent_engines.get(resource_id)
session = agent.create_session(user_id=user_id)
print("Session:", session["id"])

def print_part(part, idx):
    print(f"-- part {idx} keys:", list(part.keys()))
    if "text" in part:
        print(f"  text: {part['text']!r}")
    if "function_call" in part:
        print(f"  function_call: {part['function_call']!r}")
    if "function_response" in part:
        print(f"  function_response: {part['function_response']!r}")
    if "function_call_result" in part:
        print(f"  function_call_result: {part['function_call_result']!r}")

try:
    for event in agent.stream_query(
        user_id=user_id,
        session_id=session["id"],
        message=prompt,
    ):
        print("=== event ===")
        print({k: v for k, v in event.items() if k != "content"})
        parts = event.get("content", {}).get("parts", [])
        for i, part in enumerate(parts, start=1):
            print_part(part, i)
finally:
    agent.delete_session(user_id=user_id, session_id=session["id"])
    print("Session closed.")


Session: 1425124849516281856
=== event ===
{'finish_reason': 'STOP', 'usage_metadata': {'candidates_token_count': 16, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 16}], 'prompt_token_count': 825, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 825}], 'thoughts_token_count': 265, 'total_token_count': 1106, 'traffic_type': 'ON_DEMAND'}, 'avg_logprobs': -1.4790383577346802, 'invocation_id': 'e-fb15c7f3-7c3f-4873-9e6c-071867113478', 'author': 'academic_coordinator', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}, 'requested_tool_confirmations': {}}, 'long_running_tool_ids': [], 'id': '93ccbec5-2381-4ec0-a516-073adb81c80a', 'timestamp': 1759932124.534642}
-- part 1 keys: ['thought_signature', 'function_call']
  function_call: {'id': 'adk-682d8867-0a76-4b88-ac27-67a15ef7018d', 'args': {'request': 'Find two recent review papers on telomere biology'}, 'name': 'academic_websearch_agent'}
Session closed.


In [ ]:
from google.colab import files

# This will open a file picker in the Colab UI
uploaded = files.upload()

# Save to /content
for filename, data in uploaded.items():
    path = f"/content/{filename}"
    with open(path, "wb") as f:
        f.write(data)
    print("Saved to:", path)

Saving Karimian et al. 2024 Science.pdf to Karimian et al. 2024 Science.pdf
Saved to: /content/Karimian et al. 2024 Science.pdf


In [ ]:
import base64
from vertexai import agent_engines

pdf_path = "/content/Karimian et al. 2024 Science.pdf"

# Encode PDF to base64
with open(pdf_path, "rb") as f:
    pdf_b64 = base64.b64encode(f.read()).decode("utf-8")

agent = agent_engines.get(resource_id)
session = agent.create_session(user_id="colab_user_pdf")
print("Session:", session["id"])

try:
    prompt = "Analyze this seminal paper and summarize its key contributions. Then find two recent reviews citing it."

    message = {
        "content": {
            "parts": [
                {"text": prompt},
                {
                    "inline_data": {
                        "mime_type": "application/pdf",
                        "data": pdf_b64
                    }
                }
            ]
        }
    }

    for event in agent.stream_query(
        user_id="colab_user_pdf",
        session_id=session["id"],
        message=message,
    ):
        if "content" in event and "parts" in event["content"]:
            for p in event["content"]["parts"]:
                if "text" in p:
                    print(p["text"])
finally:
    agent.delete_session(user_id="colab_user_pdf", session_id=session["id"])
    print("Session closed.")


Session: 5659634399151390720
Session closed.


In [ ]:
import os, base64
from vertexai import agent_engines
import vertexai

# --- config ---
PROJECT = os.environ["GOOGLE_CLOUD_PROJECT"]
LOCATION = os.environ["GOOGLE_CLOUD_LOCATION"]
BUCKET  = os.environ["GOOGLE_CLOUD_STORAGE_BUCKET"]
PDF_LOCAL = "/content/Karimian et al. 2024 Science.pdf"
PDF_BASENAME = os.path.basename(PDF_LOCAL)
GCS_URI = f"gs://{BUCKET}/{PDF_BASENAME}"

# your deployed Agent Engine full resource name:
RESOURCE_ID = "projects/instr-cs795-fall25-hqin-1/locations/us-central1/reasoningEngines/8912856759031300096"
USER_ID = "colab_user_pdf"

vertexai.init(project=PROJECT, location=LOCATION, staging_bucket=f"gs://{BUCKET}")

# --- 0) upload the PDF to your staging bucket ---
print("Uploading to:", GCS_URI)
!gsutil -q cp "$PDF_LOCAL" "$GCS_URI"
print("Uploaded.")

# --- 1) open agent + session ---
agent = agent_engines.get(RESOURCE_ID)
session = agent.create_session(user_id=USER_ID)
print("Session:", session["id"])

def show(event):
    # tiny pretty-printer so you see tool calls & text
    author = event.get("author")
    fin = event.get("finish_reason")
    parts = event.get("content", {}).get("parts", [])
    print(f"\n--- event from {author}  finish={fin} ---")
    for i,p in enumerate(parts,1):
        if "text" in p:
            print(f"[text {i}] {p['text']}")
        if "function_call" in p:
            print(f"[function_call {i}] {p['function_call']}")
        if "function_response" in p:
            print(f"[function_response {i}] {p['function_response']}")
        if "function_call_result" in p:
            print(f"[function_call_result {i}] {p['function_call_result']}")

try:
    # --- 2) turn 1: greet / who-are-you (matches README) ---
    for event in agent.stream_query(
        user_id=USER_ID,
        session_id=session["id"],
        message="who are you",
    ):
        show(event)

    # --- 3) turn 2: tell it what paper you're sending (title) ---
    title_msg = "Karimian et al. 2024 Science – I’m enclosing the PDF next."
    for event in agent.stream_query(
        user_id=USER_ID,
        session_id=session["id"],
        message=title_msg,
    ):
        show(event)

    # --- 4) turn 3: send ONLY the PDF as a file attachment part ---
    message_with_pdf = {
        "content": {
            "parts": [
                {
                    "file_data": {
                        "mime_type": "application/pdf",
                        "file_uri": GCS_URI
                    }
                }
            ]
        }
    }

    for event in agent.stream_query(
        user_id=USER_ID,
        session_id=session["id"],
        message=message_with_pdf,
    ):
        show(event)

    # --- 5) optional: ask it to proceed ---
    followup = "Please analyze the seminal paper and then find two recent review papers that cite it."
    for event in agent.stream_query(
        user_id=USER_ID,
        session_id=session["id"],
        message=followup,
    ):
        show(event)

finally:
    agent.delete_session(user_id=USER_ID, session_id=session["id"])
    print("\nSession closed.")


Uploading to: gs://instr-cs795-fall25-hqin-1-tstilwel/Karimian et al. 2024 Science.pdf
Uploaded.
Session: 8632010153215918080

--- event from academic_coordinator  finish=STOP ---
[text 1] I am an AI Research Assistant. My primary function is to analyze a seminal paper you provide and then help you explore the recent academic landscape evolving from it.

Here's how I can help you:

1.  **Analyze a Seminal Paper:** I will process the paper and provide you with a detailed summary, including its key innovations and arguments.
2.  **Find Recent Citing Papers:** I will then search for recent academic papers that have cited the seminal work, giving you an overview of the latest research in the field.
3.  **Suggest Future Research Directions:** Based on the seminal paper and the recent citing papers, I will suggest potential future research directions, helping you identify gaps and promising new avenues of inquiry.

To get started, please provide the seminal paper you wish to analyze as a PDF

In [ ]:
import os
from vertexai import agent_engines
import vertexai

# Set these env vars before running (already in your .env or Colab):
# GOOGLE_CLOUD_PROJECT, GOOGLE_CLOUD_LOCATION, GOOGLE_CLOUD_STORAGE_BUCKET
project = os.environ["GOOGLE_CLOUD_PROJECT"]
location = os.environ["GOOGLE_CLOUD_LOCATION"]
bucket = os.environ["GOOGLE_CLOUD_STORAGE_BUCKET"]

vertexai.init(project=project, location=location, staging_bucket=f"gs://{bucket}")

# Use the resource_id from your deploy.py --create step
RESOURCE_ID = "projects/instr-cs795-fall25-hqin-1/locations/us-central1/reasoningEngines/8912856759031300096"
USER_ID = "colab_user_1"

agent = agent_engines.get(RESOURCE_ID)
session = agent.create_session(user_id=USER_ID)
print("Session:", session["id"])

def stream_and_print(message):
    for event in agent.stream_query(
        user_id=USER_ID,
        session_id=session["id"],
        message=message,
    ):
        if "content" in event and "parts" in event["content"]:
            for part in event["content"]["parts"]:
                if "text" in part:
                    print(part["text"])

# --- Interaction sequence ---
# 1. Greeting
stream_and_print("who are you")

# 2. Provide the title/identifier of the seminal paper
stream_and_print("Karimian et al. 2024 Science")

# 3. Provide the actual PDF as file_data (must be in your GCS bucket)
GCS_URI = "gs://instr-cs795-fall25-hqin-1-tstilwel/Karimian et al. 2024 Science.pdf"

message_with_pdf = {
    "content": {
        "parts": [
            {
                "file_data": {
                    "mime_type": "application/pdf",
                    "file_uri": GCS_URI,
                }
            }
        ]
    }
}
stream_and_print(message_with_pdf)

# 4. Ask for the analysis
stream_and_print("Please analyze this seminal paper and summarize its key contributions, then find two recent review papers citing it.")

# Clean up session when finished
agent.delete_session(user_id=USER_ID, session_id=session["id"])
print("Session closed.")


Session: 4551748890818248704
I am an AI Research Assistant. My purpose is to help you explore the academic landscape surrounding a seminal paper. I can analyze the paper, find recent works that cite it, and suggest potential future research directions.

To get started, please provide the seminal paper you would like me to analyze as a PDF.
I will now search for recent papers that cite the seminal work "Karimian et al. 2024 Science".
Based on the seminal paper from Karimian et al. and the recent citing papers, here are the papers that have cited the seminal work:

### Recent Papers Citing "Human telomere length is chromosome end-specific and conserved across individuals"

**1. High resolution long-read telomere sequencing reveals dynamic mechanisms in aging and cancer**
*   **Author(s):** Schmidt, T. T., Tyer, C., Rughani, P., et al.
*   **Publication Year:** 2024
*   **Source:** *Nature Communications*
*   **Link:** [https://doi.org/10.1038/s41467-024-48917-7](https://doi.org/10.1038/s

In [9]:
# Interactive
import os
from vertexai import agent_engines
import vertexai

# --- Config (same as your working snippet) ---
project = os.environ["GOOGLE_CLOUD_PROJECT"]
location = os.environ["GOOGLE_CLOUD_LOCATION"]
bucket = os.environ["GOOGLE_CLOUD_STORAGE_BUCKET"]

vertexai.init(project=project, location=location, staging_bucket=f"gs://{bucket}")

RESOURCE_ID = "projects/instr-cs795-fall25-hqin-1/locations/us-central1/reasoningEngines/8912856759031300096"  # <-- yours
USER_ID = "colab_user_1"

agent = agent_engines.get(RESOURCE_ID)
session = agent.create_session(user_id=USER_ID)
print("Session:", session["id"])

# ---------- Helpers ----------
def show(event):
    """Pretty-print text, tool calls, and tool responses."""
    parts = event.get("content", {}).get("parts", [])
    for p in parts:
        if "text" in p:
            print("Assistant:", p["text"])
        if "function_call" in p:
            print("[function_call]", p["function_call"])
        if "function_response" in p:
            print("[function_response]", p["function_response"])
        if "function_call_result" in p:
            print("[function_call_result]", p["function_call_result"])

def send(message):
    """Send a message (str or dict with content.parts) and stream results."""
    for ev in agent.stream_query(
        user_id=USER_ID, session_id=session["id"], message=message
    ):
        show(ev)

def attach_pdf(gcs_uri: str):
    """Send a PDF as its own turn using file_data (GCS URI)."""
    msg = {
        "content": {
            "parts": [
                {
                    "file_data": {
                        "mime_type": "application/pdf",
                        "file_uri": gcs_uri,
                    }
                }
            ]
        }
    }
    send(msg)

print(
    "\nREPL ready. Type your prompt and press Enter.\n"
    "Special commands:\n"
    "  /attach gs://bucket/path/file.pdf   -> attach a PDF\n"
    "  /quit or /exit                       -> end and close session\n"
    "Tip: do it like the README:\n"
    "  1) who are you\n"
    "  2) <paper title>\n"
    "  3) /attach gs://…/your.pdf\n"
    "  4) ask for analysis\n"
)

try:
    while True:
        msg = input("You: ").strip()
        if not msg:
            continue
        if msg.lower() in ("/quit", "/exit"):
            break
        if msg.lower().startswith("/attach "):
            gcs_uri = msg.split(" ", 1)[1].strip()
            attach_pdf(gcs_uri)
            continue

        # Normal text turn
        send(msg)

finally:
    agent.delete_session(user_id=USER_ID, session_id=session["id"])
    print("Session closed.")


Session: 8117473895788838912

REPL ready. Type your prompt and press Enter.
Special commands:
  /attach gs://bucket/path/file.pdf   -> attach a PDF
  /quit or /exit                       -> end and close session
Tip: do it like the README:
  1) who are you
  2) <paper title>
  3) /attach gs://…/your.pdf
  4) ask for analysis

You: 1
Assistant: Hello! I am an AI Research Assistant.

I can help you explore the academic landscape evolving from a seminal paper. To begin, please provide the seminal paper you wish to analyze as a PDF.
You: 2
Assistant: Hello! I am an AI Research Assistant.

I can help you explore the academic landscape evolving from a seminal paper. To begin, please provide the seminal paper you wish to analyze as a PDF.
You: 3
Assistant: Hello! I am an AI Research Assistant.

I can help you explore the academic landscape evolving from a seminal paper. To begin, please provide the seminal paper you wish to analyze as a PDF.
You: 4
Assistant: It seems like you might be having

In [10]:
import os
from vertexai import agent_engines
import vertexai

# --- Config ---
project = os.environ["GOOGLE_CLOUD_PROJECT"]
location = os.environ["GOOGLE_CLOUD_LOCATION"]
bucket = os.environ["GOOGLE_CLOUD_STORAGE_BUCKET"]

vertexai.init(project=project, location=location, staging_bucket=f"gs://{bucket}")

RESOURCE_ID = "projects/instr-cs795-fall25-hqin-1/locations/us-central1/reasoningEngines/8912856759031300096"
USER_ID = "colab_user_1"

agent = agent_engines.get(RESOURCE_ID)
session = agent.create_session(user_id=USER_ID)
print("Session:", session["id"])

def show(event):
    parts = event.get("content", {}).get("parts", [])
    for p in parts:
        if "text" in p:
            print("Assistant:", p["text"])
        if "function_call" in p:
            print("[function_call]", p["function_call"])

def send(message):
    for ev in agent.stream_query(
        user_id=USER_ID, session_id=session["id"], message=message
    ):
        show(ev)

def attach_pdf(gcs_uri: str):
    msg = {
        "content": {
            "parts": [
                {
                    "file_data": {
                        "mime_type": "application/pdf",
                        "file_uri": gcs_uri,
                    }
                }
            ]
        }
    }
    send(msg)

print(
    "\nREPL ready. Commands:\n"
    "  /attach gs://bucket/path/file.pdf   attach a PDF already in GCS\n"
    "  /quit                               end session\n"
)

try:
    while True:
        msg = input("You: ").strip()
        if not msg:
            continue
        if msg.lower() in ("/quit", "/exit"):
            break
        if msg.lower().startswith("/attach "):
            gcs_uri = msg.split(" ", 1)[1].strip()
            attach_pdf(gcs_uri)
            continue
        send(msg)
finally:
    agent.delete_session(user_id=USER_ID, session_id=session["id"])
    print("Session closed.")


Session: 5487371713404469248

REPL ready. Commands:
  /attach gs://bucket/path/file.pdf   attach a PDF already in GCS
  /quit                               end session

You: who are you?
Assistant: I am Gemini, a large language model built by Google. I am acting as an AI Research Assistant.

My primary function is to help you explore the academic landscape evolving from a seminal paper. Here's how I can help:

1.  **Analyze a Seminal Paper:** You provide a paper, and I'll analyze it to establish a foundational context.
2.  **Find Recent Citing Papers:** I will then search for recent academic papers that have cited the seminal work.
3.  **Suggest Future Research Directions:** Based on the seminal paper and the recent citing works, I will suggest potential future research directions.

To begin, please provide the seminal paper you wish to analyze as a PDF.
You: /attach gs://instr-cs795-fall25-hqin-1-tstilwel/Karimian et al. 2024 Science.pdf
You: Please analyze this seminal paper and summ

In [11]:
import os, urllib.parse
from vertexai import agent_engines
import vertexai

# --- Config ---
PROJECT = os.environ["GOOGLE_CLOUD_PROJECT"]
LOCATION = os.environ["GOOGLE_CLOUD_LOCATION"]
BUCKET  = os.environ["GOOGLE_CLOUD_STORAGE_BUCKET"]

vertexai.init(project=PROJECT, location=LOCATION, staging_bucket=f"gs://{BUCKET}")

RESOURCE_ID = "projects/instr-cs795-fall25-hqin-1/locations/us-central1/reasoningEngines/8912856759031300096"
USER_ID = "colab_user_1"

agent = agent_engines.get(RESOURCE_ID)
session = agent.create_session(user_id=USER_ID)
print("Session:", session["id"])

def show(event):
    parts = event.get("content", {}).get("parts", [])
    for p in parts:
        if "text" in p:
            print("Assistant:", p["text"])
        if "function_call" in p:
            print("[function_call]", p["function_call"])
        if "function_response" in p:
            print("[function_response]", p["function_response"])

def send(message):
    for ev in agent.stream_query(user_id=USER_ID, session_id=session["id"], message=message):
        show(ev)

def attach_pdf(gcs_uri_raw: str):
    # URL-encode spaces etc. in the path part of the URI
    # e.g., gs://bucket/Karimian et al. 2024 Science.pdf -> gs://bucket/Karimian%20et%20al.%202024%20Science.pdf
    if not gcs_uri_raw.startswith("gs://"):
        print("⚠️ Please provide a gs:// URI.")
        return
    # split "gs://bucket/path..."
    _, rest = gcs_uri_raw.split("gs://", 1)
    bucket, _, path = rest.partition("/")
    encoded_path = urllib.parse.quote(path, safe="/")  # keep slashes
    gcs_uri = f"gs://{bucket}/{encoded_path}"

    # 1) send file_data with role=user
    file_msg = {
        "role": "user",
        "content": {
            "parts": [
                {
                    "file_data": {
                        "mime_type": "application/pdf",
                        "file_uri": gcs_uri,
                    }
                }
            ]
        }
    }
    print(f"(Attaching file) {gcs_uri}")
    send(file_msg)

    # 2) follow with "enclosed" like the README
    send("enclosed")

print(
    "\nREPL ready. Commands:\n"
    "  /attach gs://bucket/path/file.pdf   attach a PDF from GCS (handles spaces)\n"
    "  /quit                               end session\n"
)

try:
    while True:
        msg = input("You: ").strip()
        if not msg:
            continue
        if msg.lower() in ("/quit", "/exit"):
            break
        if msg.lower().startswith("/attach "):
            gcs_uri = msg.split(" ", 1)[1].strip()
            attach_pdf(gcs_uri)
            continue
        send(msg)
finally:
    agent.delete_session(user_id=USER_ID, session_id=session["id"])
    print("Session closed.")


Session: 4676723780477779968

REPL ready. Commands:
  /attach gs://bucket/path/file.pdf   attach a PDF from GCS (handles spaces)
  /quit                               end session

You: who are you?
Assistant: I am Gemini, a large language model built by Google. I am acting as an AI Research Assistant.

My primary function is to help you explore the academic landscape evolving from a seminal paper you provide. Here's how I can help:

1.  **Analyze a Seminal Paper:** You provide a paper, and I'll analyze it to establish a foundational context.
2.  **Find Recent Citing Papers:** I will then search for recent academic papers that have cited the seminal work.
3.  **Suggest Future Research Directions:** Based on the original paper and the recent citing works, I will suggest potential avenues for future research.

To begin, please provide the seminal paper you would like me to analyze.
You: Karimian et al. 2024 Science
[function_call] {'id': 'adk-9aa6cb6e-d1d2-4033-a462-cb2240ce9d03', 'args':